In [1]:
import os

In [ ]:
# Check CPU information to decide the thread and chunk
# Số lõi CPU
cpu_count = os.cpu_count()
print(f"Số lõi CPU: {cpu_count}")

Số lõi CPU: 10


In [14]:
# Kiểm tra kích thước trung bình của một dòng dữ liệu:
input_file = '/Users/giakhanh/Desktop/Data Engineer/glamira_project/project/glamira.json'

# Đọc một số dòng đầu tiên và tính kích thước trung bình
def estimate_line_size(file_path, num_lines=1000):
    total_size = 0
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            total_size += len(line.encode('utf-8'))
            if i + 1 >= num_lines:
                break
    return total_size / num_lines

avg_line_size = estimate_line_size(input_file)
print(f"Kích thước trung bình mỗi dòng: {avg_line_size:.2f} bytes")

Kích thước trung bình mỗi dòng: 798.08 bytes


In [16]:
# Ước lượng chunk
available_ram = 32 * 1024**3  # Ví dụ: 2GB RAM khả dụng (thay số này theo máy bạn)
chunk_size = int(available_ram / avg_line_size / 2)  # Sử dụng nửa RAM khả dụng
print(f"Kích thước mỗi chunk (dòng): {chunk_size}")

Kích thước mỗi chunk (dòng): 21526473


In [3]:
import json
import ijson

In [4]:
# Stream JSON file to avoid loading the entire file into memory
input_file = '/Users/giakhanh/Desktop/Data Engineer/glamira_project/project/glamira.json'
output_file = '/Users/giakhanh/Desktop/Data Engineer/glamira_project/project/upload_to_gcs/output.txt'

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for i, line in enumerate(infile):
        try:
            obj = json.loads(line)  # Parse từng dòng JSON
            outfile.write(json.dumps(obj, indent=4))  # Ghi object JSON dưới dạng string
            outfile.write('\n')  # Thêm dòng mới
            if i >= 10:  # Giới hạn ghi 10 dòng đầu tiên (nếu cần)
                break
        except json.JSONDecodeError as e:
            outfile.write(f"Lỗi ở dòng {i + 1}: {e}\n")
            break

print(f"Output đã được lưu vào {output_file}")

Output đã được lưu vào /Users/giakhanh/Desktop/Data Engineer/glamira_project/project/upload_to_gcs/output.txt


In [7]:
import json

In [6]:
def find_and_print_first_json_recursive(folder_path):
    try:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith(".json"):
                    print(f"Found JSON file: {os.path.join(root, file)}")
                    return  # Dừng sau khi tìm thấy file JSON đầu tiên
    except Exception as e:
        print(f"An error occurred: {e}")

# Đường dẫn tới folder cần kiểm tra
folder_path = "/Users/giakhanh/Desktop/Data Engineer/glamira_project/project/IP_Glamira_data"
find_and_print_first_json_recursive(folder_path)


Found JSON file: /Users/giakhanh/Desktop/Data Engineer/glamira_project/project/IP_Glamira_data/79_50_119_237.json


In [ ]:
def print_json_file(file_path):
    try:
        # Mở file JSON
        with open(file_path, 'r', encoding='utf-8') as json_file:
            try:
                data = json.load(json_file)  # Parse nội dung JSON
                print("Content of the JSON file:")
                print(json.dumps(data, indent=4))  # In nội dung JSON (đẹp mắt)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
    except FileNotFoundError:
        print("File not found. Please check the path.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Đường dẫn tới file JSON cần in
file_path = "/Users/giakhanh/Desktop/Data Engineer/glamira_project/project/mappingmapping_glamira_IP.json"
print_json_file(file_path)

Content of the JSON file:


In [20]:
def print_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as json_file:
            content = json_file.read()
            try:
                # Tách từng JSON object và parse từng cái
                objects = content.split("\n")  # Giả sử mỗi JSON object nằm trên một dòng
                for idx, obj in enumerate(objects):
                    if obj.strip():  # Loại bỏ dòng trống
                        data = json.loads(obj.strip())  # Parse từng JSON object
                        print(f"Content of JSON object {idx + 1}:")
                        print(json.dumps(data, indent=4))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
    except FileNotFoundError:
        print("File not found. Please check the path.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Đường dẫn tới file JSON cần in
file_path = "/Users/giakhanh/Desktop/Data Engineer/glamira_project/project/mappingmapping_glamira_IP.json"
print_json_file(file_path)

Error decoding JSON: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)


In [18]:
from concurrent.futures import ThreadPoolExecutor

In [19]:
# Đường dẫn thư mục chứa file JSON và file đầu ra
input_directory = '/Users/giakhanh/Desktop/Data Engineer/glamira_project/project/IP_Glamira_data'
output_file = '/Users/giakhanh/Desktop/Data Engineer/glamira_project/project/mapping/unique_ips.json'

# Hàm xử lý một nhóm file
def process_files(files):
    ips = []
    for file in files:
        if file.endswith('.json'):  # Chỉ lấy file JSON
            ip_name = os.path.splitext(file)[0]  # Bỏ phần mở rộng
            ip_name_with_dots = ip_name.replace('_', '.')  # Thay _ thành .
            ips.append(ip_name_with_dots)
    return ips

# Lấy danh sách file JSON từ thư mục
all_files = os.listdir(input_directory)

# Chia file thành nhiều nhóm để xử lý song song
chunk_size = len(all_files) // os.cpu_count()  # Tùy thuộc vào số CPU
chunks = [all_files[i:i + chunk_size] for i in range(0, len(all_files), chunk_size)]

# Dùng ThreadPoolExecutor để xử lý song song
unique_ips = []
with ThreadPoolExecutor() as executor:
    results = executor.map(process_files, chunks)
    for ips in results:
        unique_ips.extend(ips)

# Lưu unique IPs vào file JSON
with open(output_file, 'w') as outfile:
    json.dump(unique_ips, outfile, indent=4)

print(f"Đã lưu danh sách unique IPs (với dấu .) vào: {output_file}")

Đã lưu danh sách unique IPs (với dấu .) vào: /Users/giakhanh/Desktop/Data Engineer/glamira_project/project/mapping/unique_ips.json


In [1]:
import pandas as pd

In [10]:
data = pd.read_csv("/Users/giakhanh/Desktop/Data Engineer/glamira_project/project/crawl_data/product_name.csv")
data

KeyboardInterrupt: 

In [7]:
print(data.info)

<bound method DataFrame.info of                         _id  time_stamp               ip  \
0  5ed8cb2bc671fc36b74653ad  1591266092    37.170.17.183   
1  5ed8cb2b4c9ffc36e828fabe  1591266092   194.193.38.240   
2  5ed8cb2cfeae1f377811165c  1591266092  212.237.237.184   
3  5ed8cb2a56b3dd3742261b8c  1591266091     121.200.6.91   
4  5ed8cb2a796d133700cefa27  1591266091    24.133.169.84   

                                          user_agent resolution  user_id_db  \
0  Mozilla/5.0 (iPhone; CPU iPhone OS 13_4_1 like...    375x667    502567.0   
1  Mozilla/5.0 (Linux; Android 10; SM-N960F) Appl...    412x846    503390.0   
2  Mozilla/5.0 (Linux; Android 9; SM-A202F) Apple...    360x780         NaN   
3  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4...   834x1112         NaN   
4  Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...  1680x1050         NaN   

                              device_id  api_version  store_id  \
0  beb2cacb-20af-4f05-9c03-c98e54a1b71a          1.0        12   
